In [ ]:
# Imports
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score , confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# Traitement des Données
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def generate_batch(image):
    train_data = []
    train_labels = []
    image_normalized = image.astype('float32') / 255.0
    image_reshaped = np.expand_dims(image_normalized, axis=-1)
    image_batch = np.expand_dims(image_reshaped, axis=0)
    
    augmented_count = 0
    for batch in datagen.flow(image_batch, batch_size=1):
        if augmented_count >= 10: 
            break
        
        augmented_image = batch[0]
        augmented_image = (augmented_image.squeeze() * 255).astype('uint8')
        
        train_data.append(augmented_image)
        train_labels.append(1)
        augmented_count += 1

    return train_data, train_labels

In [ ]:
# Initialisation des Données
def load_data():
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    # Données oliwer
    for photos in ['oliwer']:
        train = f'processed/{photos}/train/' 

        for file in os.listdir(train):
            if file.endswith('.jpg') or file.endswith('.png'):
                img = os.path.join(train, file) 
                image = cv2.imread(img, cv2.IMREAD_GRAYSCALE) 
                image = cv2.resize(image, (96, 96)) 
                
                train_data.append(image)
                train_labels.append(1) 
                
                augmented_data, augmented_labels = generate_batch(image)
                train_data.extend(augmented_data)
                train_labels.extend(augmented_labels)

        for file in os.listdir(f'processed/{photos}/test/'):
            if file.endswith('.jpg') or file.endswith('.png'):
                img = os.path.join(f'processed/{photos}/test/', file) 
                image = cv2.imread(img, cv2.IMREAD_GRAYSCALE) 
                image = cv2.resize(image, (96, 96))
                test_data.append(image)
                test_labels.append(1) 

    # Données Aléatoires
    random_train_dir = './processed/non_oliwer/train'  
    random_test_dir = './processed/non_oliwer/test'   
    
    # Traitement des images d'entraînement aléatoires (limité à 240)
    random_train_files = [f for f in os.listdir(random_train_dir) if f.endswith('.jpg') or f.endswith('.png')]
    for file in random_train_files[:2400]:
        img = os.path.join(random_train_dir, file)
        image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (96, 96))
        train_data.append(image)  
        train_labels.append(0)   

    # Traitement des images de test aléatoires (limité à 60)
    random_test_files = [f for f in os.listdir(random_test_dir) if f.endswith('.jpg') or f.endswith('.png')]
    for file in random_test_files[:600]:
        img = os.path.join(random_test_dir, file)
        image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (96, 96))
        test_data.append(image)  
        test_labels.append(0)  

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

In [ ]:
# Préparation des Données 
train_data, train_labels, test_data, test_labels = load_data() 

train_data = train_data.astype('float32') / 255.0
test_data = test_data.astype('float32') / 255.0

train_data = np.expand_dims(train_data, axis=-1)
test_data = np.expand_dims(test_data, axis=-1)

In [ ]:
# Callback
callbacks_fixed = [
    TensorBoard(log_dir='logs', histogram_freq=1),
    ModelCheckpoint('model2.keras', save_best_only=True),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
]

# Conception du Model 
model = Sequential([
    Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', input_shape=(96,96,1)),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#%load_ext tensorboard
#%tensorboard --logdir logs

In [ ]:
# Entraînement
history = model.fit(train_data, train_labels, epochs=50, validation_data=(test_data, test_labels), batch_size=32, callbacks=callbacks)

In [ ]:
# Visualisation
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Test du Modèle
predictions = model.predict(test_data)
predicted_labels = (predictions > 0.5).astype(int).flatten()
print(classification_report(test_labels, predicted_labels, target_names=label_encoder.classes_))
print("F1 Score:", f1_score(test_labels, predicted_labels, average='weighted'))
print("Confusion Matrix:\n", confusion_matrix(test_labels, predicted_labels))

# Plotting the confusion matrix
import seaborn as sns
conf_matrix = confusion_matrix(test_labels, predicted_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()  

In [ ]:
# Test
val = model.evaluate(test_data, test_labels)

load_model = keras.models.load_model('model.keras')
evaluation = load_model.evaluate(test_data, test_labels)

image = cv2.imread('test3.jpg', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (96, 96))

image = image.astype('float32') / 255.0
image = np.expand_dims(image, axis=-1)  
image = np.expand_dims(image, axis=0)   

prediction = load_model.predict(image)
confidence = prediction[0][0]

if confidence > 0.5:
    print(f"C'est Oliwer avec {confidence:.2%} de confiance")
else:
    print(f"Ce n'est pas Oliwer avec {(1-confidence):.2%} de confiance")